# Total personal income
https://www.abs.gov.au/census/find-census-data/geopackages?release=2021&geography=AUST&table=G17&gda=GDA94

In [3]:
from pathlib import Path

import pandas as pd

import geopandas as gpd
from geopandas.geodataframe import GeoDataFrame

DATA_DIR = Path("../data").resolve()

filepath = DATA_DIR / "00_raw" / "nationalmaps" / "Census_2021_G17_Total_personal_income_weekly_by_age_by_sex_Main_Statistical_Areas_Level_2_and_up_SA2_.txt"
metadata_filepath = DATA_DIR / "00_raw" / "nationalmaps" / "metadata"
digital_boundaries_filepath = metadata_filepath / "SA2_2021_AUST_GDA2020.shp"
variable_mapping = metadata_filepath = metadata_filepath / "variable_indices_mapping.json"
save_path = DATA_DIR / "01_interim" / "nationalmaps" / "Census_2021_G17_Total_personal_income_weekly_by_age_by_sex_Main_Statistical_Areas_Level_2_and_up_SA2_.shp"

data = pd.read_csv(filepath)
data_dtypes = data.dtypes.to_dict() # Store data types for later
digital_boundaries = gpd.read_file(digital_boundaries_filepath)
digital_boundaries: GeoDataFrame

assert data.REGION.dtype == 'int64'

def project_to_from_cea(df: GeoDataFrame, column: str) -> GeoDataFrame:
    """Project coordinates to and from CEA, to account for shape of earth"""
    df.loc[:,column] = gpd.GeoSeries(df.loc[:,column].values.to_crs('+proj=cea'))
    return df

digital_boundaries = project_to_from_cea(digital_boundaries, 'geometry')

assert len(set(data.REGION_TYPE)) == 1, "One region code for this file"
region_code = data.REGION_TYPE[0]

# Cast data.REGION to the same type as digital_boundaries.REGION
data.REGION =data.REGION.astype(type(digital_boundaries.SA2_CODE21[0]))
digital_boundaries = digital_boundaries[digital_boundaries.SA2_CODE21.isin(data.REGION)]

# Append geometry to data, using the SA2 code as the index on REGION
data = GeoDataFrame(data)
data = data.merge(digital_boundaries.loc[:,[f'{region_code}_CODE21', 'geometry', f'{region_code}_NAME21', 'STE_NAME21']], left_on='REGION', right_on=f'{region_code}_CODE21')


# Save geopandas
for n in data.columns:
    if type(data[n][0]) != str and not n.startswith('geometry'):
        data[n] = data[n].astype(str)
data.to_file(save_path, driver='ESRI Shapefile', index=False)

/tmp/ipykernel_1261/1736967409.py:46: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  data.to_file(save_path, driver='ESRI Shapefile', index=False)


In [4]:
import pandas as pd
import json

# Load the JSON file
with open("/mnt/c/Users/ssch7/repos/prop-recommender/data/00_raw/nationalmaps/metadata/variable_indices_mapping.json") as f:
    mapping = json.load(f)

# if SEXP is not in the mapping, use a default value to replace it

# Create a new column in your dataframe using the mapping
# data['gender'] = data['SEXP'].map(mapping['SEXP'])
data['gender'] = data['SEXP'].map(mapping['SEXP']).fillna('NA')

data

,DATAFLOW,SEXP,INCP,AGEP,REGION,REGION_TYPE,STATE,TIME_PERIOD,OBS_VALUE,SA2_CODE21,geometry,SA2_NAME21,STE_NAME21,gender
0,ABS:C21_G17_SA2(1.0.0),3,_T,_T,506031125,SA2,5,2021,8344,506031125,"POLYGON ((12901642.205 -3368559.765, 12901662....",Canning Vale - West,Western Australia,NA
1,ABS:C21_G17_SA2(1.0.0),3,_T,_T,601061035,SA2,6,2021,8238,601061035,"MULTIPOLYGON (((16423544.676 -4313898.977, 164...",Sorell - Richmond,Tasmania,NA
2,ABS:C21_G17_SA2(1.0.0),3,_T,_T,103021066,SA2,1,2021,3037,103021066,"POLYGON ((16441517.374 -3540923.164, 16439932....",Grenfell,New South Wales,NA
3,ABS:C21_G17_SA2(1.0.0),3,_T,_T,201011005,SA2,2,2021,5759,201011005,"POLYGON ((16012385.687 -3873379.668, 16012390....",Buninyong,Victoria,NA
4,ABS:C21_G17_SA2(1.0.0),3,_T,_T,108031161,SA2,1,2021,379,108031161,"MULTIPOLYGON (((17707590.595 -3313942.502, 177...",Lord Howe Island,New South Wales,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,ABS:C21_G17_SA2(1.0.0),3,_T,_T,115011294,SA2,1,2021,5284,115011294,"POLYGON ((16807530.756 -3520292.534, 16807538....",Glenhaven,New South Wales,NA
2468,ABS:C21_G17_SA2(1.0.0),3,_T,_T,801041122,SA2,8,2021,1687,801041122,"POLYGON ((16604282.838 -3656214.579, 16604250....",Throsby,Australian Capital Territory,NA
2469,ABS:C21_G17_SA2(1.0.0),3,_T,_T,402031175,SA2,4,2021,14964,402031175,"POLYGON ((15430874.042 -3625958.802, 15430861....",Northgate - Northfield,South Australia,NA
2470,ABS:C21_G17_SA2(1.0.0),3,_T,_T,505021094,SA2,5,2021,6610,505021094,"POLYGON ((12885443.851 -3348266.323, 12885436....",Trigg - North Beach - Watermans Bay,Western Australia,NA


In [23]:
len(data.SA2_CODE21.unique())

2472

In [5]:
import pandas as pd
import json

# Load the JSON file
with open("/mnt/c/Users/ssch7/repos/prop-recommender/data/00_raw/nationalmaps/metadata/variable_indices_mapping.json") as f:
    mapping = json.load(f)

mapping['SEXP'] = {int(k):v for k,v in mapping['SEXP'].items()}

# Create a new column in your dataframe using the mapping
data['gender'] = data['SEXP'].map(mapping['SEXP'])

# Print the updated dataframe
data['SEXP'].map(mapping['SEXP'])
type(data['SEXP'][0])

str

In [24]:
import pandas as pd
import json

# Load the JSON file
with open("/mnt/c/Users/ssch7/repos/prop-recommender/data/00_raw/nationalmaps/metadata/variable_indices_mapping.json") as f:
    mapping = json.load(f)

mapping['SEXP'] = {int(k):v for k,v in mapping['SEXP'].items()}

# Create a new column in your dataframe using the mapping
data['gender'] = data['SEXP'].map(mapping['SEXP'])

# Print the updated dataframe
data['SEXP'].map(mapping['SEXP'])
type(data['SEXP'][0])

str

In [25]:
data.head()

,DATAFLOW,SEXP,INCP,AGEP,REGION,REGION_TYPE,STATE,TIME_PERIOD,OBS_VALUE,SA2_CODE21,geometry,SA2_NAME21,STE_NAME21,gender
0,ABS:C21_G17_SA2(1.0.0),3,_T,_T,506031125,SA2,5,2021,8344,506031125,"POLYGON ((12901642.205 -3368559.765, 12901662....",Canning Vale - West,Western Australia,NaN
1,ABS:C21_G17_SA2(1.0.0),3,_T,_T,601061035,SA2,6,2021,8238,601061035,"MULTIPOLYGON (((16423544.676 -4313898.977, 164...",Sorell - Richmond,Tasmania,NaN
2,ABS:C21_G17_SA2(1.0.0),3,_T,_T,103021066,SA2,1,2021,3037,103021066,"POLYGON ((16441517.374 -3540923.164, 16439932....",Grenfell,New South Wales,NaN
3,ABS:C21_G17_SA2(1.0.0),3,_T,_T,201011005,SA2,2,2021,5759,201011005,"POLYGON ((16012385.687 -3873379.668, 16012390....",Buninyong,Victoria,NaN
4,ABS:C21_G17_SA2(1.0.0),3,_T,_T,108031161,SA2,1,2021,379,108031161,"MULTIPOLYGON (((17707590.595 -3313942.502, 177...",Lord Howe Island,New South Wales,NaN


In [26]:
data.to_file(save_path, driver='ESRI Shapefile', index=False)

/tmp/ipykernel_1261/583046375.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  data.to_file(save_path, driver='ESRI Shapefile', index=False)


In [41]:
len(data.SA2_CODE21.unique())

2472

In [27]:
from pathlib import Path

import pandas as pd


df = pd.read_csv("/mnt/c/Users/ssch7/repos/prop-recommender/data/00_raw/nationalmaps/Census_2021_G17_Total_personal_income_weekly_by_age_by_sex_Main_Statistical_Areas_Level_2_and_up_SA2_.txt")

In [28]:
df[['SEXP','TIME_PERIOD','INCP','AGEP','REGION_TYPE','STATE']].value_counts()

SEXP  TIME_PERIOD  INCP  AGEP  REGION_TYPE  STATE
3     2021         _T    _T    SA2          1        644
                                            3        548
                                            2        524
                                            5        267
                                            4        176
                                            8        136
                                            6        101
                                            7         70
                                            9          6
Name: count, dtype: int64

In [29]:
df.head()

,DATAFLOW,SEXP,INCP,AGEP,REGION,REGION_TYPE,STATE,TIME_PERIOD,OBS_VALUE
0,ABS:C21_G17_SA2(1.0.0),3,_T,_T,506031125,SA2,5,2021,8344
1,ABS:C21_G17_SA2(1.0.0),3,_T,_T,601061035,SA2,6,2021,8238
2,ABS:C21_G17_SA2(1.0.0),3,_T,_T,103021066,SA2,1,2021,3037
3,ABS:C21_G17_SA2(1.0.0),3,_T,_T,201011005,SA2,2,2021,5759
4,ABS:C21_G17_SA2(1.0.0),3,_T,_T,108031161,SA2,1,2021,379


### Shapefile

In [30]:
import geopandas as gpd

shapefile = gpd.read_file("/mnt/c/Users/ssch7/repos/prop-recommender/data/00_raw/nationalmaps/metadata/SA2_2021_AUST_GDA2020.shp")
shapefile.head()

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."


In [31]:
# Cast shapefile.SA2_CODE21 to the same type as df.REGION
data_dtypes = df.dtypes.to_dict()
# shapefile.SA2_CODE21.astype('category')[0], df.REGION.astype('category')[0]
df.REGION = df.REGION.astype(type(shapefile.SA2_CODE21[0]))

assert type(shapefile.SA2_CODE21[0]) == type(df.REGION[0])

In [32]:

# remove all rows in shapefile where SA2_CODE21 is not in df.REGION
shapefile = shapefile[shapefile.SA2_CODE21.isin(df.REGION)]

## GPS coordinates need projecting to and from CEA, to account for shape of the earth (check this)

In [33]:
# shapefile.loc[shapefile.SA2_NAME21=='Jervis Bay', :].geometry.centroid
shapefile.geometry = shapefile.to_crs('+proj=cea').geometry.centroid.to_crs(shapefile.crs)

In [34]:
type(shapefile)

geopandas.geodataframe.GeoDataFrame

In [35]:
display(shapefile.head(1))
shapefile.columns

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (149.79356 -35.45400)


Index(['SA2_CODE21', 'SA2_NAME21', 'CHG_FLAG21', 'CHG_LBL21', 'SA3_CODE21',
       'SA3_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'GCC_CODE21', 'GCC_NAME21',
       'STE_CODE21', 'STE_NAME21', 'AUS_CODE21', 'AUS_NAME21', 'AREASQKM21',
       'LOCI_URI21', 'geometry'],
      dtype='object')

In [36]:
df.merge(shapefile.loc[:,['SA2_CODE21', 'geometry', 'SA2_NAME21']], left_on='REGION', right_on='SA2_CODE21')

,DATAFLOW,SEXP,INCP,AGEP,REGION,REGION_TYPE,STATE,TIME_PERIOD,OBS_VALUE,SA2_CODE21,geometry,SA2_NAME21
0,ABS:C21_G17_SA2(1.0.0),3,_T,_T,506031125,SA2,5,2021,8344,506031125,POINT (115.90641 -32.08849),Canning Vale - West
1,ABS:C21_G17_SA2(1.0.0),3,_T,_T,601061035,SA2,6,2021,8238,601061035,POINT (147.56364 -42.73562),Sorell - Richmond
2,ABS:C21_G17_SA2(1.0.0),3,_T,_T,103021066,SA2,1,2021,3037,103021066,POINT (148.02595 -33.87899),Grenfell
3,ABS:C21_G17_SA2(1.0.0),3,_T,_T,201011005,SA2,2,2021,5759,201011005,POINT (143.88078 -37.64385),Buninyong
4,ABS:C21_G17_SA2(1.0.0),3,_T,_T,108031161,SA2,1,2021,379,108031161,POINT (159.07681 -31.55253),Lord Howe Island
...,...,...,...,...,...,...,...,...,...,...,...,...
2467,ABS:C21_G17_SA2(1.0.0),3,_T,_T,115011294,SA2,1,2021,5284,115011294,POINT (150.99885 -33.70210),Glenhaven
2468,ABS:C21_G17_SA2(1.0.0),3,_T,_T,801041122,SA2,8,2021,1687,801041122,POINT (149.16179 -35.18897),Throsby
2469,ABS:C21_G17_SA2(1.0.0),3,_T,_T,402031175,SA2,4,2021,14964,402031175,POINT (138.63253 -34.85287),Northgate - Northfield
2470,ABS:C21_G17_SA2(1.0.0),3,_T,_T,505021094,SA2,5,2021,6610,505021094,POINT (115.75878 -31.86512),Trigg - North Beach - Watermans Bay


In [38]:
df.REGION[0], shapefile.SA2_CODE21[0]

('506031125', '101021007')

In [39]:
int(shapefile.SA2_CODE21[0]) in list(df.REGION)

False

In [40]:
shapefile.SA2_CODE21
df.REGION.apply(lambda x: OBS_VALUE)

NameError: name 'OBS_VALUE' is not defined